<a href="https://colab.research.google.com/github/femketenharkel/Predicting_Ratings/blob/main/Baseline2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Google colab version
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Load df Google Colab (feature engineered, encoded, scaled/unscaled)
import pandas as pd
df_final = pd.read_csv("/content/drive/My Drive/Thesis/Data/df_final.csv")

pd.set_option('display.max_columns', None)
df_final.head()

# **FINAL RF BASELINE! -> 0.41**

In [ ]:
# Google colab version
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
df_final = pd.read_csv("/content/drive/My Drive/Thesis/Data/df_sampled_unscaled.csv")

MessageError: Error: credential propagation was unsuccessful

In [ ]:
# Remove total ratings per user because that is over the 100% dataset
df_final = df_final.drop(columns=['Total_ratings_per_user'])

In [ ]:
# Create the new column with the time difference
df_final['Time_release_to_rating'] = df_final['Year'] - df_final['Release_year']

# Create count variables over the 40% dataset
# Creating total ratings per movie
df_final['Total_ratings_per_movie'] = df_final.groupby('MovieID')['MovieID'].transform('count')

# Create total ratings per user
df_final['Total_ratings_per_user'] = df_final.groupby('UserID')['UserID'].transform('count')


In [ ]:
df_final.head()

,UserID,MovieID,Rating,Year,Month,Day,Hour,Age,Release_year,Female,...,Favourite_Musical,Favourite_Mystery,Favourite_Romance,Favourite_Sci-Fi,Favourite_Thriller,Favourite_War,Favourite_Western,Time_release_to_rating,Total_ratings_per_movie,Total_ratings_per_user
0,1645,485,3,2000,11,23,12,1,1993,True,...,True,False,False,False,False,False,False,7,203,228
1,3112,357,4,2000,9,20,20,1,1994,False,...,False,False,False,False,False,False,False,6,482,86
2,2077,2792,2,2000,11,19,19,1,1982,False,...,False,False,False,False,False,False,False,18,219,270
3,868,2279,4,2000,11,26,23,5,1998,False,...,False,True,False,False,False,False,False,2,84,30
4,2773,1103,4,2000,11,1,10,3,1955,False,...,False,False,False,False,False,False,False,45,160,34


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold, train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
# Define feature and target
X = df_final.drop('Rating', axis=1)
y = df_final['Rating']

# Split data 80% train, 20% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    stratify=y, random_state=42)

# Define the parameter grid for hyperparameter optimization
param_dist = {
    'n_estimators': [100, 200, 250, 300],
    'max_depth': [20, 30, 40, 50],
    'min_samples_split': [2,5,10],
    'min_samples_leaf': [1, 2, 4]
}

# Define the inner and outer cross-validation strategies
inner_cv = KFold(n_splits=3, shuffle=True, random_state=42)
outer_cv = KFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
# Initialize the RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=RandomForestClassifier(),
                                   param_distributions=param_dist, n_iter=15,
                                   cv=inner_cv, scoring='accuracy',
                                   random_state=42)

# Perform nested cross-validation
outer_scores = []
val_scores = []
best_params_list = []

for train_idx, val_idx in outer_cv.split(X_train, y_train):
    # Split the data into training and validation sets
    X_train_fold = X_train.iloc[train_idx]
    X_val_fold = X_train.iloc[val_idx]
    y_train_fold = y_train.iloc[train_idx]
    y_val_fold = y_train.iloc[val_idx]

    # Fit the model on the training fold
    random_search.fit(X_train_fold, y_train_fold)
    best_model = random_search.best_estimator_

    # Save the best parameters for this fold
    best_params_list.append(random_search.best_params_)

    # Evaluate on the validation set
    val_predictions = best_model.predict(X_val_fold)
    val_acc = accuracy_score(y_val_fold, val_predictions)
    val_prec = precision_score(y_val_fold, val_predictions, average='weighted')
    val_rec = recall_score(y_val_fold, val_predictions, average='weighted')
    val_f1 = f1_score(y_val_fold, val_predictions, average='weighted')

    # Append validation metrics
    val_scores.append((val_acc, val_prec, val_rec, val_f1))

    # Store the outer fold score (accuracy)
    outer_scores.append(random_search.best_score_)

# Print the performance of the nested cross-validation
print(f"Nested Cross-Validation Accuracy: {np.mean(outer_scores):.2f} ± {np.std(outer_scores):.2f}")

# Print validation set performance for each fold
val_scores = np.array(val_scores)
print("Validation Set Performance:")
print(f"Accuracy: {val_scores[:, 0].mean():.2f} ± {val_scores[:, 0].std():.2f}")
print(f"Precision: {val_scores[:, 1].mean():.2f} ± {val_scores[:, 1].std():.2f}")
print(f"Recall: {val_scores[:, 2].mean():.2f} ± {val_scores[:, 2].std():.2f}")
print(f"F1 Score: {val_scores[:, 3].mean():.2f} ± {val_scores[:, 3].std():.2f}")

# Print the best parameters found during hyperparameter tuning
print("Best parameters found during hyperparameter tuning:")
for i, params in enumerate(best_params_list):
    print(f"Fold {i+1}: {params}")

# Fit the best model found on the entire training set
random_search.fit(X_train, y_train)
best_model = random_search.best_estimator_

# Evaluate on the test set
test_predictions = best_model.predict(X_test)
print("Test Set Performance:")
print(f"Accuracy: {accuracy_score(y_test, test_predictions):.2f}")
print(f"Precision: {precision_score(y_test, test_predictions, average='weighted'):.2f}")
print(f"Recall: {recall_score(y_test, test_predictions, average='weighted'):.2f}")
print(f"F1 Score: {f1_score(y_test, test_predictions, average='weighted'):.2f}")

# Print the best parameters of the best-performing model
print("Best Parameters of the Best-Performing Model:")
print(random_search.best_params_)


Nested Cross-Validation Accuracy: 0.41 ± 0.00
Validation Set Performance:
Accuracy: 0.42 ± 0.00
Precision: 0.42 ± 0.00
Recall: 0.42 ± 0.00
F1 Score: 0.39 ± 0.00
Best parameters found during hyperparameter tuning:
Fold 1: {'n_estimators': 250, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_depth': 40}
Fold 2: {'n_estimators': 250, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_depth': 40}
Fold 3: {'n_estimators': 250, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_depth': 40}
Fold 4: {'n_estimators': 300, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_depth': 30}
Fold 5: {'n_estimators': 250, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_depth': 40}
Test Set Performance:
Accuracy: 0.42
Precision: 0.43
Recall: 0.42
F1 Score: 0.39
Best Parameters of the Best-Performing Model:
{'n_estimators': 250, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_depth': 40}


# Gradient boosting test

In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV, KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
# Split the data into features (X) and target (y)
X = df_final.drop('Rating', axis=1)
y = df_final['Rating']

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    random_state=42)

In [ ]:
# Define the Gradient Boosting model
model = GradientBoostingClassifier()

In [ ]:
# Define the hyperparameter grid for Random Search
param_dist = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

In [ ]:
# Inner cross-validation loop for hyperparameter tuning (3 folds)
inner_cv = KFold(n_splits=3, shuffle=True, random_state=1)

# Outer cross-validation loop for model evaluation (5 folds)
outer_cv = KFold(n_splits=5, shuffle=True, random_state=1)

# Define classifier
clf = RandomizedSearchCV(estimator=model, param_distributions=param_dist,
                         cv=inner_cv, n_iter=50, random_state=42)

In [ ]:
# Initialize lists to store metrics
accuracies = []
precisions = []
recalls = []
f1s = []

In [ ]:
# Perform nested cross-validation
for train_index, val_index in outer_cv.split(X_train):
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

    clf.fit(X_train_fold, y_train_fold)

    # Get the best model and predict on validation data
    best_model = clf.best_estimator_
    y_val_pred = best_model.predict(X_val_fold)

    # Calculate evaluation metrics
    accuracies.append(accuracy_score(y_val_fold, y_val_pred))
    precisions.append(precision_score(y_val_fold, y_val_pred, average='weighted'))
    recalls.append(recall_score(y_val_fold, y_val_pred, average='weighted'))
    f1s.append(f1_score(y_val_fold, y_val_pred, average='weighted'))

# Fit the final model on the entire training data and evaluate on the test set
best_clf = clf.fit(X_train, y_train)
y_test_pred = best_clf.predict(X_test)

# Print the results of the nested cross-validation
print(f'Nested CV accuracy (validation data): {np.mean(accuracies):.2f} ± {np.std(accuracies):.2f}')
print(f'Nested CV precision (validation data): {np.mean(precisions):.2f} ± {np.std(precisions):.2f}')
print(f'Nested CV recall (validation data): {np.mean(recalls):.2f} ± {np.std(recalls):.2f}')
print(f'Nested CV F1-score (validation data): {np.mean(f1s):.2f} ± {np.std(f1s):.2f}')

# Print test set results
print(f'Test set accuracy: {accuracy_score(y_test, y_test_pred):.2f}')
print(f'Test set precision: {precision_score(y_test, y_test_pred, average="weighted"):.2f}')
print(f'Test set recall: {recall_score(y_test, y_test_pred, average="weighted"):.2f}')
print(f'Test set F1-score: {f1_score(y_test, y_test_pred, average="weighted"):.2f}')

# Print the best parameters from hyperparameter tuning
print(f'Best parameters found: {clf.best_params_}')


# **Gradient boosting baseline (same style to RF)**

In [17]:
# Google colab version
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
df_final = pd.read_csv("/content/drive/My Drive/Thesis/Data/df_final.csv")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
df_final.head()

,UserID,MovieID,Rating,Year,Month,Day,Hour,Age,Release_year,Female,...,Favourite_Musical,Favourite_Mystery,Favourite_Romance,Favourite_Sci-Fi,Favourite_Thriller,Favourite_War,Favourite_Western,Time_release_to_rating,Total_ratings_per_movie,Total_ratings_per_user
0,1645,485,3,2000,11,23,12,1,1993,True,...,True,False,False,False,False,False,False,7,203,228
1,3112,357,4,2000,9,20,20,1,1994,False,...,False,False,False,False,False,False,False,6,482,86
2,2077,2792,2,2000,11,19,19,1,1982,False,...,False,False,False,False,False,False,False,18,219,270
3,868,2279,4,2000,11,26,23,5,1998,False,...,False,True,False,False,False,False,False,2,84,30
4,2773,1103,4,2000,11,1,10,3,1955,False,...,False,False,False,False,False,False,False,45,160,34


In [16]:
# This output is with the 1% data for testing

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import KFold, RandomizedSearchCV
from sklearn.model_selection import KFold, train_test_split, RandomizedSearchCV
import numpy as np

# Define feature and target
X = df_final.drop('Rating', axis=1)
y = df_final['Rating']

# Split data 80% train, 20% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    stratify=y, random_state=42)
# Define the parameter grid for tuning
param_dist = {
    'n_estimators': [50, 100, 150, 200],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'max_depth': [3, 7, 10, 20]
}

# Define the inner and outer cross-validation strategies
inner_cv = KFold(n_splits=3, shuffle=True, random_state=42)
outer_cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Initialize the RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=GradientBoostingClassifier(),
    param_distributions=param_dist,
    n_iter=15,
    cv=inner_cv,
    scoring='accuracy',
    random_state=42,
    n_jobs=-1  # Utilize all available cores for parallel processing
)

# Perform nested cross-validation
outer_scores = []
val_scores = []
best_params_list = []

for train_idx, val_idx in outer_cv.split(X_train, y_train):
    # Split the data into training and validation sets
    X_train_fold = X_train.iloc[train_idx]
    X_val_fold = X_train.iloc[val_idx]
    y_train_fold = y_train.iloc[train_idx]
    y_val_fold = y_train.iloc[val_idx]

    # Fit the model on the training fold
    random_search.fit(X_train_fold, y_train_fold)
    best_model = random_search.best_estimator_

    # Save the best parameters for this fold
    best_params_list.append(random_search.best_params_)

    # Evaluate on the validation set
    val_predictions = best_model.predict(X_val_fold)
    val_acc = accuracy_score(y_val_fold, val_predictions)
    val_prec = precision_score(y_val_fold, val_predictions, average='weighted')
    val_rec = recall_score(y_val_fold, val_predictions, average='weighted')
    val_f1 = f1_score(y_val_fold, val_predictions, average='weighted')

    # Append validation metrics
    val_scores.append((val_acc, val_prec, val_rec, val_f1))

    # Store the outer fold score (accuracy)
    outer_scores.append(random_search.best_score_)

# Print the performance of the nested cross-validation
print(f"Nested Cross-Validation Accuracy: {np.mean(outer_scores):.2f} ± {np.std(outer_scores):.2f}")

# Print validation set performance for each fold
val_scores = np.array(val_scores)
print("Validation Set Performance:")
print(f"Accuracy: {val_scores[:, 0].mean():.2f} ± {val_scores[:, 0].std():.2f}")
print(f"Precision: {val_scores[:, 1].mean():.2f} ± {val_scores[:, 1].std():.2f}")
print(f"Recall: {val_scores[:, 2].mean():.2f} ± {val_scores[:, 2].std():.2f}")
print(f"F1 Score: {val_scores[:, 3].mean():.2f} ± {val_scores[:, 3].std():.2f}")

# Print the best parameters found during hyperparameter tuning
print("Best parameters found during hyperparameter tuning:")
for i, params in enumerate(best_params_list):
    print(f"Fold {i+1}: {params}")

# Fit the best model found on the entire training set
random_search.fit(X_train, y_train)
best_model = random_search.best_estimator_

# Evaluate on the test set
test_predictions = best_model.predict(X_test)
print("Test Set Performance:")
print(f"Accuracy: {accuracy_score(y_test, test_predictions):.2f}")
print(f"Precision: {precision_score(y_test, test_predictions, average='weighted'):.2f}")
print(f"Recall: {recall_score(y_test, test_predictions, average='weighted'):.2f}")
print(f"F1 Score: {f1_score(y_test, test_predictions, average='weighted'):.2f}")

# Print the best parameters of the best-performing model
print("Best Parameters of the Best-Performing Model:")
print(random_search.best_params_)


/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_divisi

Nested Cross-Validation Accuracy: 0.34 ± 0.03
Validation Set Performance:
Accuracy: 0.34 ± 0.08
Precision: 0.25 ± 0.15
Recall: 0.34 ± 0.08
F1 Score: 0.26 ± 0.11
Best parameters found during hyperparameter tuning:
Fold 1: {'n_estimators': 50, 'max_depth': 20, 'learning_rate': 0.01}
Fold 2: {'n_estimators': 50, 'max_depth': 3, 'learning_rate': 0.01}
Fold 3: {'n_estimators': 50, 'max_depth': 3, 'learning_rate': 0.01}
Fold 4: {'n_estimators': 50, 'max_depth': 3, 'learning_rate': 0.01}
Fold 5: {'n_estimators': 50, 'max_depth': 3, 'learning_rate': 0.01}
Test Set Performance:
Accuracy: 0.40
Precision: 0.25
Recall: 0.40
F1 Score: 0.28
Best Parameters of the Best-Performing Model:
{'n_estimators': 50, 'max_depth': 3, 'learning_rate': 0.01}


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
